<a href="https://colab.research.google.com/github/erindakapllani/AppliedMath_Task2/blob/main/final_NPL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
from google.colab import files
uploaded = files.upload()

Saving 8314-e.pdf to 8314-e (2).pdf
Saving BT31-4-17-2008E.pdf to BT31-4-17-2008E (2).pdf
Saving CC171-5-2007E.pdf to CC171-5-2007E.pdf
Saving CC171-5-2008E.pdf to CC171-5-2008E.pdf
Saving CC171-11-2009-eng.pdf to CC171-11-2009-eng.pdf
Saving CC171-11-2010-eng.pdf to CC171-11-2010-eng.pdf


In [10]:
from google.colab import files
import os

# Upload files
uploaded = files.upload()

# Define a folder name for storing uploaded files
folder_name = 'Nuclear_Safety'

# Create the folder if it doesn't exist
if not os.path.exists(folder_name):
    os.makedirs(folder_name)

# Move each uploaded file into the folder
for filename in uploaded.keys():
    # Get the content of the file
    file_content = uploaded[filename]

    # Define the path to store the file
    file_path = os.path.join(folder_name, filename)

    # Write the file content to the specified path
    with open(file_path, 'wb') as f:
        f.write(file_content)

    print(f"File '{filename}' has been saved to '{folder_name}' folder.")

Saving 8314-e.pdf to 8314-e (3).pdf
Saving BT31-4-17-2008E.pdf to BT31-4-17-2008E (3).pdf
Saving CC171-5-2007E.pdf to CC171-5-2007E (1).pdf
Saving CC171-5-2008E.pdf to CC171-5-2008E (1).pdf
Saving CC171-11-2009-eng.pdf to CC171-11-2009-eng (1).pdf
File '8314-e (3).pdf' has been saved to 'Nuclear_Safety' folder.
File 'BT31-4-17-2008E (3).pdf' has been saved to 'Nuclear_Safety' folder.
File 'CC171-5-2007E (1).pdf' has been saved to 'Nuclear_Safety' folder.
File 'CC171-5-2008E (1).pdf' has been saved to 'Nuclear_Safety' folder.
File 'CC171-11-2009-eng (1).pdf' has been saved to 'Nuclear_Safety' folder.


In [11]:
import os

# Define the folder path where you moved the files
folder_path = '/content/Nuclear_Safety'

# List all files in the folder
files_in_folder = os.listdir(folder_path)

# Print the list of files
print(f"Files in '{folder_path}' folder:")
for file_name in files_in_folder:
    print(file_name)


Files in '/content/Nuclear_Safety' folder:
CC171-5-2007E (1).pdf
8314-e (1).pdf
8314-e (3).pdf
CC171-11-2009-eng (1).pdf
BT31-4-17-2008E (3).pdf
BT31-4-17-2008E (1).pdf
CC171-5-2008E (1).pdf


In [7]:
pip install PyMuPDF


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 65.5 MB/s eta 0:00:00


In [33]:
import os
import fitz  # PyMuPDF
import spacy
from spacy.matcher import Matcher
from google.colab import files  # Added for file download


In [34]:
# Load the SpaCy model
nlp = spacy.load("en_core_web_sm")

In [35]:
# Define the path to the folder containing the PDF documents
folder_path = '/content/Nuclear_Safety'

In [36]:
# Function to read all text from PDF files in the folder
def read_documents(folder_path):
    documents = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.pdf'):
            pdf_path = os.path.join(folder_path, filename)
            doc = fitz.open(pdf_path)
            text = ""
            for page_num in range(len(doc)):
                page_text = doc[page_num].get_text()
                text += page_text
            documents.append(text)
    return documents

In [39]:
def generate_rule_based_questions(doc):
    matcher = Matcher(nlp.vocab)

    # Define patterns specific to the nuclear industry
    patterns = [
        [{"LOWER": "nuclear"}, {"LOWER": "reactor"}],
        [{"LOWER": "nuclear"}, {"LOWER": "fission"}],
        [{"LOWER": "uranium"}],
        [{"LOWER": "pressurized"}, {"LOWER": "water"}, {"LOWER": "reactor"}],
        [{"LOWER": "safety"}, {"LOWER": "measures"}],
        [{"LOWER": "radiation"}, {"LOWER": "leaks"}],
        [{"LOWER": "international"}, {"LOWER": "atomic"}, {"LOWER": "energy"}, {"LOWER": "agency"}],
        [{"LOWER": "nuclear"}, {"LOWER": "waste"}],
        [{"LOWER": "spent"}, {"LOWER": "fuel"}],
        [{"LOWER": "radioactive"}, {"LOWER": "material"}],
        [{"LOWER": "reactor"}, {"LOWER": "cooling"}, {"LOWER": "system"}],
        [{"LOWER": "emergency"}, {"LOWER": "response"}],
        [{"LOWER": "regulatory"}, {"LOWER": "requirements"}],
        [{"LOWER": "safety"}, {"LOWER": "protocols"}],
        [{"LOWER": "critical"}, {"LOWER": "mass"}],
        [{"LOWER": "nuclear"}, {"LOWER": "power"}, {"LOWER": "plant"}],
        [{"LOWER": "fuel"}, {"LOWER": "assembly"}],
        [{"LOWER": "radiation"}, {"LOWER": "monitoring"}],
    ]


    # Adding patterns to the matcher
    for i, pattern in enumerate(patterns):
        matcher.add(f"NUCLEAR_PATTERN_{i}", [pattern])

    # Processing the document and extracting QA pairs
    qa_pairs = []
    seen_questions = set()
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        question = f"What is {span.text}?"
        answer = span.sent.text
        if question not in seen_questions:
            qa_pairs.append((question, answer))
            seen_questions.add(question)

    return qa_pairs

In [40]:
# Main function to process documents and generate QA pairs
def main():
    documents = read_documents(folder_path)
    all_qa_pairs = []

    for document in documents:
        doc = nlp(document)
        qa_pairs = generate_rule_based_questions(doc)
        all_qa_pairs.extend(qa_pairs)

    # Output the QA pairs to a file
    output_file = 'QA_pairs_output.txt'
    with open(output_file, 'w') as f:
        for question, answer in all_qa_pairs:
            f.write(f"Question: {question}\nAnswer: {answer}\n\n")
    print(f"Generated QA pairs are saved in '{output_file}'.")

    # Download the output file
    files.download(output_file)

# Run the main function
if __name__ == "__main__":
    main()

Generated QA pairs are saved in 'QA_pairs_output.txt'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
from google.colab import files

# Download the output file
files.download('QA_pairs_output.txt')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>